In [1]:
import PyPDF2

# Index the data

## Convert the pdf into text files to maintiain the line and spacing

#### Difference in pages can be used for splitbypages

In [2]:
document = None
with open("test-data/pdfs/32008R1099.pdf", "rb") as pdf:
    reader = PyPDF2.PdfReader(pdf)

    for page_number, page in enumerate(reader.pages):
        if page_number == 0:
            document = page.extract_text()
        document += "\n====================\n" + page.extract_text()
        # document += page.extract_text()

with open("test-data/txts/32008R1099-paged-formated-PyPDF2.txt", "w", encoding="utf-8") as textfile:
    textfile.write(document)

#### No difference in pages can be used for splitbylines

In [3]:
document = None
with open("test-data/pdfs/32008R1099.pdf", "rb") as pdf:
    reader = PyPDF2.PdfReader(pdf)

    for page_number, page in enumerate(reader.pages):
        if page_number == 0:
            document = page.extract_text()
        # document += "\n====================\n" + page.extract_text()
        document += page.extract_text()

with open("test-data/txts/32008R1099-formated-PyPDF2.txt", "w", encoding="utf-8") as textfile:
    textfile.write(document)

## Get the text files and convert them to chunks

#### Chunk by pages

In [4]:
with open("test-data/txts/32008R1099-paged-formated-PyPDF2.txt", "rb") as testfile:
    document = testfile.read().decode("utf-8")

    for page_number, page in enumerate(document.split("\n====================\n")):
        with open(f"test-data/chunks/1p-32008R1099/{page_number}.txt", "w") as chunkfile:
            chunkfile.write(page)

#### Chunk by number of lines

In [1]:
with open("test-data/txts/32008R1099-formated-PyPDF2.txt", "rb") as testfile:
    document = testfile.read().decode("utf-8")

    chunk = ''
    for line_number, line in enumerate(document.splitlines(keepends=True)):
        if line_number == 0:
            chunk = line
            continue

        if line_number % 1 == 0 and not line_number == 0:
            with open(f"test-data/chunks/1l-32008R1099/{line_number}.txt", "w") as chunkfile:
                chunkfile.write(chunk)
                chunk = ''

        chunk += line

- spacy_textspiltter
- chunk_size and chunk_overlap

- create a AST for documents
- then embed it? :O
- check hyphenations

## Embed the chunks

In [7]:
from sentence_transformers import SentenceTransformer

/home/kushal/.config/python.env/nlptrans.env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
transformer = SentenceTransformer("flax-sentence-embeddings/all_datasets_v4_MiniLM-L6")

/home/kushal/.config/python.env/nlptrans.env/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [9]:
import os

files = os.listdir("test-data/chunks/1p-32008R1099/")

embeddings = []
for file in files:
    if os.path.isfile(f"test-data/chunks/1p-32008R1099/{file}"):
        with open(f"test-data/chunks/1p-32008R1099/{file}", "r") as file:
            embedding = transformer.encode(file.read())

    embeddings.append(embedding)

## Embed user query

In [10]:
system_prompt = """You are a legal agent of EU law of regulations. 
You will summarize the laws that are provided to you in the query and 
analyze the question and related answers then formulate a proper response to 
the query that even a lay person can understand."""

user_query = "What is this regulation for?"

In [11]:
embedded_query = transformer.encode(user_query)

## Find similar chunks to query

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [13]:
cosine_similarities = cosine_similarity(np.expand_dims(embedded_query, axis=0), embeddings)

In [14]:
sorted_similarities = sorted(cosine_similarities[0])

top_indices = []
for index, sim in enumerate(cosine_similarities[0]):
    if sim in sorted_similarities[-5:]:
        top_indices.append((index, sim))

In [15]:
for i in top_indices:
    print(i)

(13, 0.38801306)
(26, 0.49764177)
(37, 0.30940196)
(50, 0.5452769)
(54, 0.38801306)
